In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_barcelona = pd.read_csv('data_barcelona.csv')

In [3]:
data_barcelona = data_barcelona.drop(['Unnamed: 0'], axis=1)

In [4]:
from scipy import stats

# Assuming df is the name of your dataframe
z_scores = np.abs(stats.zscore(data_barcelona))

# Remove rows where any of the columns have a z-score greater than 3
data_barcelona_cleaned = data_barcelona[(z_scores < 3).all(axis=1)]

In [5]:
data_barcelona_cleaned.corr()

,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
realSum,1.000000,0.646489,-0.191883,0.374416,0.110072,0.062784,0.450178,-0.212775,-0.047858,0.220585,0.169099,0.024603,-0.065427,NaN,-0.677138,0.043846,0.040042
person_capacity,0.646489,1.000000,-0.162492,0.348268,-0.011359,-0.071620,0.616982,-0.052728,-0.024435,0.084639,0.018089,-0.035170,0.014067,NaN,-0.671827,-0.055013,-0.105857
multi,-0.191883,-0.162492,1.000000,-0.550334,-0.038589,0.028175,-0.094045,0.060266,0.022383,-0.028110,-0.034094,0.052465,0.016589,NaN,0.217021,0.061402,0.016708
biz,0.374416,0.348268,-0.550334,1.000000,-0.017995,-0.148712,0.180129,-0.183487,-0.080118,0.106983,0.138043,-0.055360,-0.100168,NaN,-0.382671,-0.087953,-0.051076
cleanliness_rating,0.110072,-0.011359,-0.038589,-0.017995,1.000000,0.645606,0.027223,0.007404,-0.002667,-0.022256,-0.017797,0.009144,0.006569,NaN,-0.015248,0.276839,0.002426
guest_satisfaction_overall,0.062784,-0.071620,0.028175,-0.148712,0.645606,1.000000,0.013072,0.039236,0.059109,-0.016744,-0.033591,0.054053,0.062528,NaN,0.064843,0.328277,0.012611
bedrooms,0.450178,0.616982,-0.094045,0.180129,0.027223,0.013072,1.000000,0.011364,0.015238,-0.007435,-0.067380,-0.044948,0.045048,NaN,-0.456477,0.040228,-0.082430
dist,-0.212775,-0.052728,0.060266,-0.183487,0.007404,0.039236,0.011364,1.000000,0.397158,-0.745695,-0.862465,0.070788,0.592745,NaN,0.078870,0.019555,0.004498
metro_dist,-0.047858,-0.024435,0.022383,-0.080118,-0.002667,0.059109,0.015238,0.397158,1.000000,-0.341229,-0.378648,0.316701,0.445237,NaN,0.031493,0.009690,0.034220
attr_index_norm,0.220585,0.084639,-0.028110,0.106983,-0.022256,-0.016744,-0.007435,-0.745695,-0.341229,1.000000,0.810192,0.100190,-0.293811,NaN,-0.089173,-0.022964,0.098303


In [35]:
data_barcelona_cleaned_reg = data_barcelona_cleaned.drop(['lng', 'lat'], axis=1)

In [36]:
x = data_barcelona_cleaned_reg.drop('realSum', axis=1)
y = data_barcelona_cleaned_reg.realSum

In [37]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
count,1901.000000,1901.000000,1901.000000,1901.000000,1901.000000,1901.000000,1901.000000,1901.000000,1901.000000,1901.000000,1901.0,1901.00000,1901.000000,1901.000000
mean,2.490268,0.395055,0.305629,9.394529,91.938979,1.083114,2.080531,0.419264,16.167493,19.259613,0.0,0.84324,0.185692,0.464492
std,1.001859,0.488991,0.460794,0.772591,6.471757,0.358996,1.314313,0.235048,7.820922,9.693090,0.0,0.36367,0.388960,0.498869
min,2.000000,0.000000,0.000000,7.000000,67.000000,0.000000,0.100000,0.010000,4.602831,5.061928,0.0,0.00000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,9.000000,88.000000,1.000000,1.080000,0.260000,10.528499,10.862599,0.0,1.00000,0.000000,0.000000
50%,2.000000,0.000000,0.000000,10.000000,93.000000,1.000000,1.720000,0.370000,13.995029,17.811252,0.0,1.00000,0.000000,0.000000
75%,2.000000,1.000000,1.000000,10.000000,97.000000,1.000000,2.900000,0.540000,21.260123,26.684620,0.0,1.00000,0.000000,1.000000
max,6.000000,1.000000,1.000000,10.000000,100.000000,2.000000,6.060000,1.280000,45.201404,49.749481,0.0,1.00000,1.000000,1.000000


In [38]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.584
Method:                 Least Squares   F-statistic:                     206.1
Date:                Thu, 09 Mar 2023   Prob (F-statistic):               0.00
Time:                        22:28:35   Log-Likelihood:                -11538.
No. Observations:                1901   AIC:                         2.310e+04
Df Residuals:                    1887   BIC:                         2.318e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [10]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [11]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.583
Method:                 Least Squares   F-statistic:                     242.4
Date:                Thu, 09 Mar 2023   Prob (F-statistic):               0.00
Time:                        15:13:04   Log-Likelihood:                -11541.
No. Observations:                1901   AIC:                         2.311e+04
Df Residuals:                    1889   BIC:                         2.317e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        260.2012      2.411    107.913      0.0

In [12]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-71.34559420819124

In [13]:
model.score(x_train, y_train)

0.5852778305744899

In [14]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [15]:
mse(y_test,y_pred)

11381.062496928265

In [16]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

106.68206267657307

In [17]:
##prediction on the train set
mse(y_train,y_pred_train)

10982.44206788879

In [18]:
mae(y_test,y_pred)

70.65223443873636

In [19]:
R2=r2_score(y_test,y_pred)
R2

0.6083180044483466

In [20]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.601391152436983

In [21]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.5852734430032258, Test -> 0.6082622031780464


In [22]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.4785498022584854, Test -> 0.4799092861377461


In [23]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 1901 rows and 11 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.49
The coefficient of determination for the train data is R2=1.00


In [24]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.18641653 0.46669719 0.17701234 0.26380982 0.48355459]
Score stats: 0.32 accuracy with a standard deviation of 0.13


In [25]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.3362261125799406, 'Linear Regression': 0.5744376720748589, 'KNN': 0.4432630547889083}


In [26]:
data_barcelona_cleaned.to_excel('data_barcelona.xlsx')